# 1. Data Preparation

This notebook covers the initial data preparation steps:
1.  Loading the dataset.
2.  Inspecting the data for quality issues.
3.  Cleaning the data (handling missing values, duplicates).
4.  Performing feature engineering.
5.  Saving the cleaned dataset for further analysis.


In [5]:
import pandas as pd

pd.set_option('display.max_columns', None)

print("Loading Hotel_bookings_final.csv")
df = pd.read_csv('Hotel_bookings_final.csv')
print("Dataset loaded successfully.")

print("\n Data Overview")
df.info()

print("\n First 5 Rows")
display(df.head())


Loading Hotel_bookings_final.csv
Dataset loaded successfully.

 Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         30000 non-null  int64  
 1   property_id         30000 non-null  int64  
 2   city                30000 non-null  object 
 3   star_rating         30000 non-null  int64  
 4   booking_date        30000 non-null  object 
 5   check_in_date       24532 non-null  object 
 6   check_out_date      24532 non-null  object 
 7   room_type           30000 non-null  object 
 8   num_rooms_booked    30000 non-null  int64  
 9   stay_type           30000 non-null  object 
 10  booking_channel     30000 non-null  object 
 11  booking_value       30000 non-null  float64
 12  costprice           30000 non-null  int64  
 13  markup              30000 non-null  int64  
 14  selling_price       3000

,customer_id,property_id,city,star_rating,booking_date,check_in_date,check_out_date,room_type,num_rooms_booked,stay_type,booking_channel,booking_value,costprice,markup,selling_price,payment_method,refund_status,refund_amount,channel_of_booking,booking_status,travel_date,cashback,coupon_redeem,Coupon USed?
0,492,3,San Francisco,4,2024-04-01,2024-05-24,2024-05-26,Standard,1,Leisure,Mobile App,19361.0,19361,5981,25342,PayPal,Yes,369.648995,Web,Confirmed,2024-03-04,5.374694,0.000000,No
1,180,3,Dallas,3,2024-04-01,2024-05-10,2024-05-17,Deluxe,1,Leisure,Mobile App,6137.0,6137,1896,8033,Bank Transfer,Yes,492.505903,Web,Confirmed,2024-07-19,7.161033,0.000000,No
2,50,5,Dallas,3,2024-04-01,2024-05-31,2024-06-05,Deluxe,1,Business,Web,22702.0,22702,7013,29715,Debit Card,Yes,0.000000,iOS,Confirmed,2024-03-22,0.000000,0.000000,No
3,294,3,Orlando,4,2024-04-01,2024-04-18,2024-04-24,Deluxe,3,Leisure,Web,34068.0,34068,10524,44592,Bank Transfer,Yes,545.541516,Android,Confirmed,2024-11-24,7.932170,24.500442,Yes
4,40,5,Seattle,5,2024-04-01,NaN,NaN,Deluxe,1,Leisure,Mobile App,12127.0,12127,3746,15873,Debit Card,Yes,211.367602,Web,Cancelled,2024-03-02,0.000000,0.000000,No


In [6]:
# Data Cleaning 

# 1. Handling Missing Values
print(" Missing Value Check")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Targeted handling: keep cancelled rows even if dates are missing; drop only non-cancelled rows missing check-in/out dates.
if ('check_in_date' in df.columns) and ('check_out_date' in df.columns) and ('booking_status' in df.columns):
    non_cancelled_mask = df['booking_status'] != 'Cancelled'
    missing_dates_mask = df['check_in_date'].isna() | df['check_out_date'].isna()
    to_drop_mask = non_cancelled_mask & missing_dates_mask
    print(f"\nDropping rows missing dates that are not cancelled: {to_drop_mask.sum()}")
    df = df.loc[~to_drop_mask].copy()
else:
    # Fallback if required columns are unavailable
    if df.isnull().sum().sum() > 0:
        df.dropna(inplace=True)
        print("\nRows with missing values have been removed (fallback).")

# 2. Handling Duplicates
print("\n Duplicate Row Check")
duplicate_rows = df.duplicated().sum()
print(f"Found {duplicate_rows} duplicate rows.")

if duplicate_rows > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicate rows have been removed.")

print(f"\nShape of the dataframe after cleaning: {df.shape}")


 Missing Value Check
check_in_date     5468
check_out_date    5468
dtype: int64

Dropping rows missing dates that are not cancelled: 423

 Duplicate Row Check
Found 0 duplicate rows.

Shape of the dataframe after cleaning: (29577, 24)


In [7]:
# Feature Engineering

# Convert date columns to proper datetime objects
df['booking_date'] = pd.to_datetime(df['booking_date'])
df['check_in_date'] = pd.to_datetime(df['check_in_date'])
df['check_out_date'] = pd.to_datetime(df['check_out_date'])

# Create new columns for analysis

# Calculate length of stay from check-in and check-out dates
df['length_of_stay'] = (df['check_out_date'] - df['check_in_date']).dt.days

# Create a binary 'is_cancelled' column from the 'booking_status'
df['is_cancelled'] = df['booking_status'].apply(lambda x: 1 if x == 'Cancelled' else 0)

# Calculate average daily rate from the total booking value and stay length
df['avg_daily_rate'] = df['booking_value'] / (df['length_of_stay'] + 1e-6)

# Extract temporal features for seasonal analysis
df['booking_year'] = df['booking_date'].dt.year
df['booking_month'] = df['booking_date'].dt.month
df['booking_day_of_week'] = df['booking_date'].dt.dayofweek

print("\nFeature engineering complete. Preview of new columns:")
display(df[['booking_date', 'length_of_stay', 'is_cancelled', 'avg_daily_rate']].head())



Feature engineering complete. Preview of new columns:


,booking_date,length_of_stay,is_cancelled,avg_daily_rate
0,2024-04-01,2.0,0,9680.495160
1,2024-04-01,7.0,0,876.714160
2,2024-04-01,5.0,0,4540.399092
3,2024-04-01,6.0,0,5677.999054
4,2024-04-01,NaN,1,NaN


In [8]:
# Save Cleaned Data

output_filename = 'hotel_bookings_cleaned.csv'
df.to_csv(output_filename, index=False)

print(f"Data preparation complete.")
print(f"Cleaned and feature-engineered data saved to '{output_filename}'.")
print(f"Final shape: {df.shape}")


Data preparation complete.
Cleaned and feature-engineered data saved to 'hotel_bookings_cleaned.csv'.
Final shape: (29577, 30)
